In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import tarfile

import shutil
import yaml

from functools import partial
from glob import glob, iglob
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from projections.shapefiles import load_shapes, iter_records
from projections.models import Records
from projections.elevation import get_indices_by_file
from projections import raster, utils


pd.set_option('max_columns', None)

In [2]:
def save_location_mapping(row_and_path):
    row, path = row_and_path
    shape = row['geometry']
    
    subset = raster.find_subset_with_intersection_area(IMAGE, shape)

    if subset.empty:
        with open(path, 'w') as f:
            f.write('')
        return

    subset['id'] = row['id']
    
    subset.to_csv(path, index=False)        

In [3]:
base_file = "Alfalfa.tif"
read_path = Path('../Data/FAO/Agro-ClimaticPotentialYield')
output_path = Path('../Output/FAO/Agro-ClimaticPotentialYield/')
partial_path = output_path / 'partial'
by_country_path = output_path / 'by_country'

output_path.mkdir(exist_ok=True)
partial_path.mkdir(exist_ok=True)
by_country_path.mkdir(exist_ok=True)

In [4]:
# Validate that all TIF share the same coordinates
x = None
y = None

for tif in read_path.glob('*.tif'):
    image = utils.read_tif(tif)
    if x is None:
        x = image.x
        y = image.y
    else:
        assert np.all(x == image.x) and np.all(y == image.y), tif
        
no_data_value = image._FillValue

# Map raster to polygons

In [5]:
geo_df = gpd.read_file('../Shapefiles/preprocessed/all_countries_with_eth.shp')

In [6]:
n_processes = 30

IMAGE = utils.read_tif(read_path / base_file)

iterator = partial(utils.yield_missing_shapes, save_path=partial_path, prefix=base_file[:-4])

if n_processes == 1:
    for row_and_path in tqdm(iterator(geo_df)):
        save_location_mapping(row_and_path)
else:
    with ThreadPoolExecutor(n_processes) as tpe:
        for _ in tqdm(
            tpe.map(save_location_mapping, iterator(geo_df)), 
            total=geo_df.shape[0]
        ):
            pass

 97%|██████████████████████████████▉ | 118888/122772 [12:55:27<25:20,  2.56it/s]


# Union portions from different files and shapes

In [7]:
df_by_region = {}
for file in tqdm(partial_path.glob('*.csv'), desc='Reading'):
    try:
        df = pd.read_csv(file)
    except pd.errors.EmptyDataError:
        continue
        
    if 'id' not in df.columns:
        df['id'] = df['adm2']
        df['id'].fillna(df['adm1'], inplace=True)
        df['id'].fillna(df['adm0'], inplace=True)
    region = df.loc[0, 'id']
    df_by_region.setdefault(region, []).append(df)

for region, dfs in tqdm(df_by_region.items(), desc='Saving'):
    df = utils.combine_dataframes(dfs)
    df.to_feather(by_country_path / f'{region}.feather')
    
del df_by_region

Reading: 122772it [02:37, 779.54it/s] 
Saving: 100%|████████████████████████████| 47728/47728 [01:35<00:00, 501.93it/s]


# Aggregate

In [8]:
utils.aggregate_feather_splits_and_save(by_country_path, output_path / base_file, no_data_value=-9999)

Grouping: 100%|██████████████████████████| 47728/47728 [04:50<00:00, 164.58it/s]


# Additional files with same raster

In [9]:
for file in read_path.glob('*.tif'):
    print(file.name)
    file_path = output_path / file.name[:-4]
    file_path.mkdir(exist_ok=True)

    IMAGE = utils.read_tif(file)
    increment = raster.get_increment_from_tif(IMAGE)
    
    for df_path in tqdm(by_country_path.glob('*.feather')):
        df = pd.read_feather(df_path)
        pol = utils.get_mock_polygon_from_df(df, increment=increment)
        subdf = raster.merge_df_to_array_by_lat_lon(df, IMAGE, pol)
        if subdf.empty:
            print(df_path.name, 'is empty')
        else:
            subdf.to_feather(file_path / df_path.name)
            
    utils.aggregate_feather_splits_and_save(file_path, output_path / file.name, no_data_value=-9999)
    shutil.rmtree(file_path)

Soy.tif


47728it [08:17, 96.02it/s] 
Grouping: 100%|██████████████████████████| 47728/47728 [04:40<00:00, 170.06it/s]


Yam.tif


47728it [07:59, 99.54it/s] 
Grouping: 100%|██████████████████████████| 47728/47728 [04:39<00:00, 170.49it/s]


Tea.tif


...

47728it [08:23, 94.85it/s] 
Grouping: 100%|██████████████████████████| 47728/47728 [05:19<00:00, 149.20it/s]


Para_Rubber_highInput.tif


47728it [08:23, 94.79it/s] 
Grouping: 100%|██████████████████████████| 47728/47728 [05:18<00:00, 149.69it/s]


Napier_Grass_highInput.tif


47728it [08:25, 94.47it/s]
Grouping: 100%|██████████████████████████| 47728/47728 [05:19<00:00, 149.36it/s]


In [10]:
for file in read_path.glob('*.tif'):
    file_path = output_path / file.name[:-4]
    shutil.rmtree(file_path)